<h1 align='center'>Segmenting and Clustering Neighborhoods in Toronto</h1>

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim 
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium

print('Libraries imported.')

Libraries imported.


In [5]:
df = pd.read_csv('CA_PC_w_latlon.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head(2)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572


<div class = "alert alert-info" style = "margin-top: 20px">Acquire the geograpical coordinate of Toronto:</div>

In [3]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<div class = "alert alert-info" style = "margin-top: 20px">Create a map of Toronto with neighborhoods superimposed on top.</div>

In [21]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude+0.06, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='',
        fill=True,
        fill_color='#7A7A9C',
        fill_opacity=1,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<div class = "alert alert-info" style = "margin-top: 20px">Now let's start utilizing the Foursquare API to explore the neighborhoods and segment them.</div>

In [22]:
CLIENT_ID = 'C3UOHBNDZTAKKMLLAXEZKBOSFCLEZAT5J5UUJ3XUJ5OEUZJB'
CLIENT_SECRET = '4TCGSEHFSEHYRTFINMQHYC3BW10230XA153FKOD3LGS5EP2L'
VERSION = '20180605'

<div class = "alert alert-info" style = "margin-top: 20px">Create a function to repeat the same process to all the neighborhoods.</div>

In [40]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT=100, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<div class = "alert alert-info" style = "margin-top: 20px">Run the above function on each neighborhood and create a new dataframe called "tv" which contains all nearby venues in Toronto neibourhoods.</div>

In [41]:
tv = getNearbyVenues(names=df['Neighbourhood'],
                     latitudes=df['Latitude'],
                     longitudes=df['Longitude']
                    )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

<div class = "alert alert-info" style = "margin-top: 20px">Reveal the size of the resulting dataframe.</div>

In [42]:
print(tv.shape)
tv.head()

(2154, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


<div class = "alert alert-info" style = "margin-top: 20px">Check how many venues were returned for each neighborhood.</div>

In [44]:
tv.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",27,27,27,27,27,27
Berczy Park,58,58,58,58,58,58
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17


<div class = "alert alert-info" style = "margin-top: 20px">Find out how many unique categories can be curated from all the returned venues.</div>

In [45]:
print('There are {} uniques categories.'.format(len(tv['Venue Category'].unique())))

There are 267 uniques categories.


<div class = "alert alert-info" style = "margin-top: 20px">Analyze Each Neighborhood</div>

In [46]:
t_oh = pd.get_dummies(tv[['Venue Category']], prefix="", prefix_sep="")
t_oh['Neighbourhood'] = tv['Neighbourhood'] 
fixed_columns = [t_oh.columns[-1]] + list(t_oh.columns[:-1])
t_oh = t_oh[fixed_columns]

t_oh.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

<div class = "alert alert-info" style = "margin-top: 20px">Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.</div>

In [47]:
t_gp = t_oh.groupby('Neighbourhood').mean().reset_index()
t_gp

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

<div class = "alert alert-info" style = "margin-top: 20px">Write a function to sort the venues in descending order..</div>

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<div class = "alert alert-info" style = "margin-top: 20px">Create the new dataframe and display the top <b>10</b> venues for each neighborhood..</div>

In [86]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = t_gp['Neighbourhood']

for ind in np.arange(t_gp.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(t_gp.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Coffee Shop,Athletics & Sports,Pub,Sandwich Place,Yoga Studio,Discount Store,Dessert Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Diner,Supermarket,Mobile Phone Shop,Sushi Restaurant,Ice Cream Shop,Restaurant,Deli / Bodega,Fried Chicken Joint
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Restaurant,Thai Restaurant,Comfort Food Restaurant,Pizza Place,Juice Bar,Japanese Restaurant,Fast Food Restaurant


<div class = "alert alert-info" style = "margin-top: 20px">Run k-means to cluster the neighborhood into <b>5</b> clusters.</div>

In [87]:
kclusters = 5

t_gc = t_gp.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(t_gc)
kmeans.labels_[:] 

array([1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,
       2, 1, 2, 1, 1, 0, 2, 1, 3, 1, 1, 1, 1, 1, 2, 1, 0, 2, 1, 1, 1, 2,
       4, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 2, 0, 1, 1, 0, 0, 2,
       1, 2, 1, 0, 1, 1, 2], dtype=int32)

<div class = "alert alert-info" style = "margin-top: 20px">Create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood.</div>

In [88]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
t_mg = df
t_mg = t_mg.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
t_mg.dropna(axis=0, how='any', inplace=True)
t_mg['Cluster Labels'] = t_mg['Cluster Labels'].astype(int)
t_mg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99 entries, 0 to 102
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Postal Code             99 non-null     object 
 1   Borough                 99 non-null     object 
 2   Neighbourhood           99 non-null     object 
 3   Latitude                99 non-null     float64
 4   Longitude               99 non-null     float64
 5   Cluster Labels          99 non-null     int64  
 6   1st Most Common Venue   99 non-null     object 
 7   2nd Most Common Venue   99 non-null     object 
 8   3rd Most Common Venue   99 non-null     object 
 9   4th Most Common Venue   99 non-null     object 
 10  5th Most Common Venue   99 non-null     object 
 11  6th Most Common Venue   99 non-null     object 
 12  7th Most Common Venue   99 non-null     object 
 13  8th Most Common Venue   99 non-null     object 
 14  9th Most Common Venue   99 non-null     obj

<div class = "alert alert-info" style = "margin-top: 20px">Visualize the resulting clusters.</div>

In [91]:
map_clusters = folium.Map(location=[latitude+0.05, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(t_mg['Latitude'], t_mg['Longitude'], t_mg['Neighbourhood'], t_mg['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<div class = "alert alert-info" style = "margin-top: 20px">
    <h3 class="alert-heading">CLUSTER 0: Gourmet Streets</h3>
    <hr>
    <p>Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, assign a name to each cluster.</p>
</div>

In [92]:
t_mg.loc[t_mg['Cluster Labels'] == 0, t_mg.columns[[1] + list(range(5, t_mg.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Portuguese Restaurant,Hockey Arena,Pizza Place,Coffee Shop,French Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
6,Scarborough,0,Print Shop,Fast Food Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
8,East York,0,Pizza Place,Gym / Fitness Center,Athletics & Sports,Gastropub,Intersection,Pet Store,Café,Breakfast Spot,Bank,Pharmacy
10,North York,0,Pizza Place,Bakery,Pub,Japanese Restaurant,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store,Dog Run
17,Etobicoke,0,Pet Store,Beer Store,Pharmacy,Pizza Place,Coffee Shop,Convenience Store,Café,Shopping Plaza,Liquor Store,Gastropub
29,East York,0,Sandwich Place,Indian Restaurant,Gym,Restaurant,Pizza Place,Pharmacy,Park,Liquor Store,Yoga Studio,Supermarket
51,Scarborough,0,Motel,American Restaurant,Skating Rink,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
55,North York,0,Sandwich Place,Italian Restaurant,Coffee Shop,Restaurant,Thai Restaurant,Comfort Food Restaurant,Pizza Place,Juice Bar,Japanese Restaurant,Fast Food Restaurant
56,York,0,Sandwich Place,Coffee Shop,Discount Store,Bar,Yoga Studio,Doner Restaurant,Diner,Distribution Center,Dog Run,Donut Shop
70,Etobicoke,0,Pizza Place,Chinese Restaurant,Intersection,Coffee Shop,Sandwich Place,Discount Store,Middle Eastern Restaurant,Yoga Studio,Dim Sum Restaurant,Diner


<div class = "alert alert-info" style = "margin-top: 20px">
    <h3 class="alert-heading">CLUSTER 1: Afternoon Tea Time</h3><br>
    </div>

In [93]:
t_mg.loc[t_mg['Cluster Labels'] == 1, t_mg.columns[[1] + list(range(5, t_mg.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Farmers Market,Restaurant
3,North York,1,Clothing Store,Accessories Store,Coffee Shop,Boutique,Miscellaneous Shop,Event Space,Arts & Crafts Store,Furniture / Home Store,Women's Store,Vietnamese Restaurant
4,Downtown Toronto,1,Coffee Shop,Diner,Yoga Studio,Café,Bar,Bank,Sushi Restaurant,Restaurant,Fried Chicken Joint,Beer Bar
7,North York,1,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Restaurant,Sporting Goods Shop,Café,Bike Shop,Dim Sum Restaurant,Sandwich Place
9,Downtown Toronto,1,Coffee Shop,Clothing Store,Italian Restaurant,Bubble Tea Shop,Japanese Restaurant,Café,Cosmetics Shop,Diner,Plaza,Pizza Place
12,Scarborough,1,Home Service,Bar,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dessert Shop
13,North York,1,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Restaurant,Sporting Goods Shop,Café,Bike Shop,Dim Sum Restaurant,Sandwich Place
14,East York,1,Pharmacy,Athletics & Sports,Curling Ice,Dance Studio,Skating Rink,Beer Store,Park,General Travel,Golf Course,Eastern European Restaurant
15,Downtown Toronto,1,Café,Coffee Shop,Clothing Store,Restaurant,American Restaurant,Cocktail Bar,Cosmetics Shop,Department Store,Creperie,Beer Bar
16,York,1,Trail,Field,Tennis Court,Hockey Arena,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


<div class = "alert alert-info" style = "margin-top: 20px">
    <h3 class="alert-heading">CLUSTER 2: Embracing Greenery</h3><br>
    </div>

In [94]:
t_mg.loc[t_mg['Cluster Labels'] == 2, t_mg.columns[[1] + list(range(5, t_mg.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
21,York,2,Park,Women's Store,Pool,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
35,East York,2,Park,Convenience Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
52,North York,2,Park,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
61,Central Toronto,2,Park,Bus Line,Swim School,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
64,York,2,Convenience Store,Park,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
66,North York,2,Park,Convenience Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
77,Etobicoke,2,Park,Sandwich Place,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
85,Scarborough,2,Park,Playground,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
91,Downtown Toronto,2,Park,Playground,Trail,Eastern European Restaurant,Electronics Store,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dance Studio


<div class = "alert alert-info" style = "margin-top: 20px">
    <h3 class="alert-heading">CLUSTER 3: Evening Exercise & Family Union</h3><br>
    </div>

In [95]:
t_mg.loc[t_mg['Cluster Labels'] == 3, t_mg.columns[[1] + list(range(5, t_mg.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,North York,3,Gym,Pizza Place,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
83,Central Toronto,3,Gym,Tennis Court,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,Deli / Bodega


<div class = "alert alert-info" style = "margin-top: 20px">
    <h3 class="alert-heading">CLUSTER 4: Meditation</h3><br>
    </div>

In [96]:
t_mg.loc[t_mg['Cluster Labels'] == 4, t_mg.columns[[1] + list(range(5, t_mg.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,4,Music Venue,Garden,Yoga Studio,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


<div class = "alert alert-danger" style = "margin-top: 20px" align="center">
    <hr>This is the end of the Segmenting and Clustering Neighbourhoods in Toronto notebook series.<br><h3 class="alert-heading">Thanks for reading! :)</h3><hr>
    </div>